# Postman Tour generic code

In [1]:
# TODO
# -- create a class of networks to store all the information generated
# -- uses logger rather than print
# -- implement unit testing / pipeline testing
# -- convert config to YAML

## Configuration

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import numpy as np
import imageio
import pytest
# import osmnx    # couldn't install this package; abandoning visualization here for now

In [3]:
config = {
    'sub_nets': {
        'A': ['A', 'all'],
        'B': ['B', 'all']
    },
    
    'start_nodes': {
        'A': "A4",
        'B': "A4"
    },
    
    'infile': "KampongGlamNetwork.xlsx",
    'edge_sht': "Edges",
    
    'method': 'path',   # DEFUNCT
}

In [4]:
network_df = pd.read_excel(config['infile'], sheet_name=config['edge_sht'])

network_df.head()

,NodeA,NodeB,distance,graph,section
0,A1,A2,100,original,A
1,A2,L1,34,original,A
2,L1,B2,34,original,A
3,B2,C3,72,original,A
4,C3,C2,45,original,A


In [5]:
config['sub_nets'].keys()

dict_keys(['A', 'B'])

In [6]:
graphs = {}
for s in config['sub_nets'].keys():
    print("Making graph: ", s)
    graphs[s] = nx.Graph()
    
    # Add edges and edge attributes
    for index, row in network_df.iterrows():
        # add the appropriate edges only
        if row['section'] in config['sub_nets'][s]:
            print("-- Adding: (" + row['NodeA'] + "," + row['NodeB'] + ")")
            graphs[s].add_edge(row['NodeA'], row['NodeB'], **row[2:].to_dict())
            
    print('# of edges in {}: {}'.format(s, graphs[s].number_of_edges()))

Making graph:  A
-- Adding: (A1,A2)
-- Adding: (A2,L1)
-- Adding: (L1,B2)
-- Adding: (B2,C3)
-- Adding: (C3,C2)
-- Adding: (C2,C1)
-- Adding: (C1,B1)
-- Adding: (B1,A1)
-- Adding: (B1,B1-2)
-- Adding: (B1-2,L1)
-- Adding: (B1-2,C2)
-- Adding: (C2,J1)
-- Adding: (J1,J2)
-- Adding: (J1,D2)
-- Adding: (D2,D1)
-- Adding: (D1,E1)
-- Adding: (E1,F1)
-- Adding: (F1,G1)
-- Adding: (F1,F2)
-- Adding: (F2,F3)
-- Adding: (F3,H3)
-- Adding: (H3,G3)
-- Adding: (H3,H2)
-- Adding: (E2,E3)
-- Adding: (E1,E2)
-- Adding: (E3,E4)
-- Adding: (E4,F3)
-- Adding: (I1,E2)
-- Adding: (K1,C3)
-- Adding: (K1,K2)
-- Adding: (K2,K3)
-- Adding: (K3,E4)
-- Adding: (C4,C5)
-- Adding: (C4,B3)
-- Adding: (B2,B3)
-- Adding: (B3,B4)
-- Adding: (B4,B5)
-- Adding: (B5,A4)
-- Adding: (A4,A3)
-- Adding: (A3,A2)
-- Adding: (L2,B4)
-- Adding: (L2,B3)
-- Adding: (L2,A3)
-- Adding: (A4,B5)
-- Adding: (I1,D3)
-- Adding: (D3,J3)
-- Adding: (J3,J2)
-- Adding: (J2,K1)
-- Adding: (D2,D3)
-- Adding: (G1,G2)
-- Adding: (G2,G3)
-- Addin

In [7]:
# Preview first 5 edges
list(graphs['A'].edges(data=True))[0:5]

[('A1', 'A2', {'distance': 100, 'graph': 'original', 'section': 'A'}),
 ('A1', 'B1', {'distance': 66, 'graph': 'original', 'section': 'A'}),
 ('A2', 'L1', {'distance': 34, 'graph': 'original', 'section': 'A'}),
 ('A2', 'A3', {'distance': 106, 'graph': 'original', 'section': 'A'}),
 ('L1', 'B2', {'distance': 34, 'graph': 'original', 'section': 'A'})]

## Optimize the path

In [8]:
def get_shortest_paths_distances(graph, pairs, edge_weight_name):
    """
    Compute shortest distance between each pair of nodes in a graph.  Return a dictionary keyed on node pairs (tuples).
    
    Uses Dijkstra's algorithm from networkx library
    
    Taken from http://brooksandrew.github.io/simpleblog/articles/intro-to-graph-optimization-solving-cpp/
    """
    distances = {}
    for pair in pairs:
        distances[pair] = nx.dijkstra_path_length(graph, pair[0], pair[1], weight=edge_weight_name)
        
    return distances

In [9]:
def create_complete_graph(pair_weights, flip_weights=True):
    """
    Create a completely connected graph using a list of vertex pairs and the shortest path distances between them
    Parameters: 
        pair_weights: list[tuple] from the output of get_shortest_paths_distances
        flip_weights: Boolean. Should we negate the edge attribute in pair_weights?
        
    Taken from http://brooksandrew.github.io/simpleblog/articles/intro-to-graph-optimization-solving-cpp/
    """
    g = nx.Graph()
    for k, v in pair_weights.items():
        wt_i = - v if flip_weights else v
        g.add_edge(k[0], k[1], **{'distance': v, 'weight': wt_i})
        
    return g

In [10]:
def add_augmenting_path_to_graph(graph, min_weight_pairs):
    """
    Add the min weight matching edges to the original graph
    Parameters:
        graph: NetworkX graph (original graph from trailmap)
        min_weight_pairs: list[tuples] of node pairs from min weight matching
    Returns:
        augmented NetworkX graph
        
    Taken from http://brooksandrew.github.io/simpleblog/articles/intro-to-graph-optimization-solving-cpp/
    """
    
    # We need to make the augmented graph a MultiGraph so we can add parallel edges
    graph_aug = nx.MultiGraph(graph.copy())
    for pair in min_weight_pairs:
        graph_aug.add_edge(pair[0], 
                           pair[1], 
                           **{'distance': nx.dijkstra_path_length(graph, pair[0], pair[1]), 'graph': 'augmented'}
                          )
        
    return graph_aug

In [11]:
def create_eulerian_circuit(graph_augmented, graph_original, starting_node=None, method='circuit'):
    """
    Create the eulerian path using only edges from the original graph.
    
    Taken from http://brooksandrew.github.io/simpleblog/articles/intro-to-graph-optimization-solving-cpp/
    """
    
    # get the first best-guess circuit.  N.B. will contain edges that don't exist in the original graph
    euler_circuit = []
    naive_circuit = list(nx.eulerian_circuit(graph_augmented, source=starting_node))
    
    for edge in naive_circuit:
        edge_data = graph_augmented.get_edge_data(edge[0], edge[1])    
        
        if edge_data[0]['graph'] != 'augmented':
            # If `edge` exists in original graph, grab the edge attributes and add to eulerian circuit.
            edge_att = graph_original[edge[0]][edge[1]]
            euler_circuit.append((edge[0], edge[1], edge_att)) 
        else: 
            # find the shortest path between true nodes
            aug_path = nx.shortest_path(graph_original, edge[0], edge[1], weight='distance')
            aug_path_pairs = list(zip(aug_path[:-1], aug_path[1:]))
            
            print('Filling in edges for augmented edge: {}'.format(edge))
            print('Augmenting path: {}'.format(' => '.join(aug_path)))
            print('Augmenting path pairs: {}\n'.format(aug_path_pairs))
            
            # If `edge` does not exist in original graph, find the shortest path between its nodes and 
            #  add the edge attributes for each link in the shortest path.
            for edge_aug in aug_path_pairs:
                edge_aug_att = graph_original[edge_aug[0]][edge_aug[1]]
                euler_circuit.append((edge_aug[0], edge_aug[1], edge_aug_att))
                                      
    return euler_circuit

In [12]:
def test_add_augmenting_path_to_graph(g):
    """
    Check the function only results in even-degree edges
    """
    
    odd_list = [n for n, d in g.degree() if d % 2 == 1]
    assert len(odd_list) == 0, "Graph not a valid graph to create Eulerian circuit"

### Find all the odd degree nodes (must be traversed multiple times)

In [13]:
nodes_odd_degree = {}

# Calculate list of nodes with odd degree
for s in config['sub_nets'].keys():
    nodes_odd_degree[s] = [v for v, d in graphs[s].degree() if d % 2 == 1]
    print('Number of nodes of odd degree for graph {}: {}'.format(s, len(nodes_odd_degree[s])))

Number of nodes of odd degree for graph A: 32
Number of nodes of odd degree for graph B: 12


### Add edges to the graph corresponding to the shortest path pairing

In [14]:
odd_node_pairs = {}

# compute all node pairs, get a list of tuples
for s in config['sub_nets'].keys():
    odd_node_pairs[s] = list(itertools.combinations(nodes_odd_degree[s], 2))

    print('Number of pairs for graph {}: {}'.format(s, len(odd_node_pairs[s])))

Number of pairs for graph A: 496
Number of pairs for graph B: 66


In [15]:
odd_node_pairs_shortest_paths = {}

# compute shortest path between node pairs
for s in config['sub_nets'].keys():
    odd_node_pairs_shortest_paths[s] = get_shortest_paths_distances(graphs[s], odd_node_pairs[s], 'distance')

In [16]:
# inspect the shortest paths for each pair
dict(list(odd_node_pairs_shortest_paths['A'].items())[0:10])

{('A2', 'L1'): 34,
 ('A2', 'C1'): 225,
 ('A2', 'B1'): 163,
 ('A2', 'J1'): 246,
 ('A2', 'J2'): 204,
 ('A2', 'D2'): 284,
 ('A2', 'D1'): 319,
 ('A2', 'E1'): 376,
 ('A2', 'F1'): 417,
 ('A2', 'F3'): 492}

In [17]:
g_odd_complete = {}

# Generate the complete graph for the odd-degree nodes
for s in config['sub_nets'].keys():
    g_odd_complete[s] = create_complete_graph(odd_node_pairs_shortest_paths[s], flip_weights=True)

    print('Number of nodes for graph {}: {}'.format(s, len(g_odd_complete[s].nodes())))
    print('Number of edges for graph {}: {}'.format(s, len(g_odd_complete[s].edges())))

Number of nodes for graph A: 32
Number of edges for graph A: 496
Number of nodes for graph B: 12
Number of edges for graph B: 66


In [18]:
odd_matching_dupes = {}

# Compute min weight matching.
# Note: max_weight_matching uses the 'weight' attribute by default as the attribute to maximize.
for s in config['sub_nets'].keys():
    odd_matching_dupes[s] = nx.algorithms.max_weight_matching(g_odd_complete[s], True)

    print('Number of edges in matching for graph {}: {}'.format(s, len(odd_matching_dupes[s])))

Number of edges in matching for graph A: 16
Number of edges in matching for graph B: 6


In [19]:
# TODO: check if you need to deduplicate this
odd_matching_dupes['A']

{('B1', 'C1'),
 ('B4', 'B5'),
 ('C4', 'C5'),
 ('C7', 'C6'),
 ('D2', 'D1'),
 ('F1', 'E1'),
 ('F3', 'E4'),
 ('G2', 'I1'),
 ('H2', 'H3'),
 ('I2', 'E3'),
 ('J2', 'J1'),
 ('J3', 'K1'),
 ('J4', 'D4'),
 ('K2', 'K3'),
 ('L1', 'A2'),
 ('L2', 'A3')}

In [20]:
g_aug = {}

# Create augmented graph: add the min weight matching edges to g
for s in config['sub_nets'].keys():
    g_aug[s] = add_augmenting_path_to_graph(graphs[s], odd_matching_dupes[s])

    # Counts
    print('Number of edges in original graph {}: {}'.format(s, len(graphs[s].edges())))
    print('Number of edges in augmented graph {}: {}'.format(s, len(g_aug[s].edges())))

Number of edges in original graph A: 65
Number of edges in augmented graph A: 81
Number of edges in original graph B: 25
Number of edges in augmented graph B: 31


In [21]:
for s in config['sub_nets'].keys():
    test_add_augmenting_path_to_graph(g_aug[s])
    pd.value_counts([e[1] for e in g_aug[s].degree()])

### Find the Eulerian tour with the revised network

In [22]:
euler_circuit = {}

# Create the Eulerian circuit, filling in unknown connections with the shortest path between nodes
for s in config['sub_nets'].keys():
    print("## Filling for graph {}".format(s))
    euler_circuit[s] = create_eulerian_circuit(g_aug[s], graphs[s], config['start_nodes'][s], method=config['method'])
    
    print('Length of Eulerian circuit for graph {}: {}'.format(s, len(euler_circuit[s])))

## Filling for graph A
Filling in edges for augmented edge: ('I1', 'G2')
Augmenting path: I1 => E2 => F2 => G2
Augmenting path pairs: [('I1', 'E2'), ('E2', 'F2'), ('F2', 'G2')]

Filling in edges for augmented edge: ('K1', 'J3')
Augmenting path: K1 => J2 => J3
Augmenting path pairs: [('K1', 'J2'), ('J2', 'J3')]

Length of Eulerian circuit for graph A: 84
## Filling for graph B
Length of Eulerian circuit for graph B: 31


# Gather stats about the route

In [23]:
# Preview first 20 directions of CPP solution
for s in config['sub_nets'].keys():
    print("Euler circuit for graph {}".format(s))
    for i, edge in enumerate(euler_circuit[s][:]):
        print(i, edge)

Euler circuit for graph A
0 ('A4', 'A3', {'distance': 118, 'graph': 'original', 'section': 'A'})
1 ('A3', 'L2', {'distance': 32, 'graph': 'original', 'section': 'A'})
2 ('L2', 'B4', {'distance': 32, 'graph': 'original', 'section': 'A'})
3 ('B4', 'B5', {'distance': 124, 'graph': 'original', 'section': 'A'})
4 ('B5', 'C7', {'distance': 81, 'graph': 'original', 'section': 'A'})
5 ('C7', 'C6', {'distance': 38, 'graph': 'original', 'section': 'A'})
6 ('C6', 'C7', {'distance': 38, 'graph': 'original', 'section': 'A'})
7 ('C7', 'K3', {'distance': 54, 'graph': 'original', 'section': 'A'})
8 ('K3', 'K2', {'distance': 85, 'graph': 'original', 'section': 'A'})
9 ('K2', 'J4', {'distance': 27, 'graph': 'original', 'section': 'A'})
10 ('J4', 'D4', {'distance': 33, 'graph': 'original', 'section': 'A'})
11 ('D4', 'J4', {'distance': 33, 'graph': 'original', 'section': 'A'})
12 ('J4', 'J3', {'distance': 121, 'graph': 'original', 'section': 'A'})
13 ('J3', 'D3', {'distance': 30, 'graph': 'original', 'sec

In [24]:
# Computing some stats
for s in config['sub_nets'].keys():
    print("Statistics for graph {}".format(s))
    total_mileage_of_circuit = sum([edge[2]['distance'] for edge in euler_circuit[s]])
    total_mileage_on_orig_map = sum(nx.get_edge_attributes(graphs[s], 'distance').values())
    _vcn = pd.value_counts(pd.value_counts([(e[0]) for e in euler_circuit[s]]), sort=False)
    node_visits = pd.DataFrame({'n_visits': _vcn.index, 'n_nodes': _vcn.values})
    _vce = pd.value_counts(pd.value_counts([sorted(e)[0] + sorted(e)[1] for e in nx.MultiDiGraph(euler_circuit[s]).edges()]))
    edge_visits = pd.DataFrame({'n_visits': _vce.index, 'n_edges': _vce.values})

    # Printing stats
    print('Mileage of circuit: {0:.2f}'.format(total_mileage_of_circuit))
    print('Mileage on original map: {0:.2f}'.format(total_mileage_on_orig_map))
    print('Mileage retracing edges: {0:.2f}'.format(total_mileage_of_circuit-total_mileage_on_orig_map))
    print('Percent of mileage retraced: {0:.2f}%\n'.format((1-total_mileage_of_circuit/total_mileage_on_orig_map)*-100))

    print('Number of edges in circuit: {}'.format(len(euler_circuit[s])))
    print('Number of edges in original graph: {}'.format(len(graphs[s].edges())))
    print('Number of nodes in original graph: {}\n'.format(len(graphs[s].nodes())))

    print('Number of edges traversed more than once: {}\n'.format(len(euler_circuit[s])-len(graphs[s].edges())))  

    print('Number of times visiting each node:')
    print(node_visits.to_string(index=False))

    print('\nNumber of times visiting each edge:')
    print(edge_visits.to_string(index=False))

Statistics for graph A
Mileage of circuit: 4991.00
Mileage on original map: 4116.00
Mileage retracing edges: 875.00
Percent of mileage retraced: 21.26%

Number of edges in circuit: 84
Number of edges in original graph: 65
Number of nodes in original graph: 44

Number of edges traversed more than once: 19

Number of times visiting each node:
 n_visits  n_nodes
        1        7
        2       34
        3        3

Number of times visiting each edge:
 n_visits  n_edges
        1       46
        2       19
Statistics for graph B
Mileage of circuit: 2011.00
Mileage on original map: 1656.00
Mileage retracing edges: 355.00
Percent of mileage retraced: 21.44%

Number of edges in circuit: 31
Number of edges in original graph: 25
Number of nodes in original graph: 19

Number of edges traversed more than once: 6

Number of times visiting each node:
 n_visits  n_nodes
        1        7
        2       12

Number of times visiting each edge:
 n_visits  n_edges
        1       19
        2    